In [84]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import panel as pn
from panel.interact import interact
from panel import widgets
pn.extension('plotly')
import hvplot.pandas
from pathlib import Path
import datetime as dt 
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv

In [85]:
load_dotenv()

True

In [86]:
# Import the CSVs RobinTrack (rt) to Pandas DataFrames
abb_path = Path("../data/cleandata/Robintrack_Data/ABB_csv_clean.csv")
amd_path = Path("../data/cleandata/Robintrack_Data/AMD_csv_clean.csv")
riot_path = Path("../data/cleandata/Robintrack_Data/RIOT_csv_clean.csv")
tsla_path = Path("../data/cleandata/Robintrack_Data/TSLA_csv_clean.csv")

abb_rt_data = pd.read_csv(abb_path, infer_datetime_format=True, parse_dates=True)
amd__rt_data = pd.read_csv(amd_path, infer_datetime_format=True, parse_dates=True)
riot_rt_data = pd.read_csv(riot_path, infer_datetime_format=True, parse_dates=True)
tsla_data = pd.read_csv(tsla_path, infer_datetime_format=True, parse_dates=True)
abb_rt_data.head()

,timestamp,users_holding
0,2020-01-01 00:46:09,2149
1,2020-01-01 01:46:06,2149
2,2020-01-01 02:46:03,2149
3,2020-01-01 03:46:09,2149
4,2020-01-01 04:45:49,2149


In [87]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("APCA_API_KEY")
alpaca_secret_key = os.getenv("APCA_SECRET_KEY")

# Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [88]:
type(alpaca_api_key)

str

In [89]:
# Format current date as ISO format
start_date = pd.Timestamp("2020-01-01", tz="America/New_York").isoformat()

end_date = pd.Timestamp("2020-08-10", tz="America/New_York").isoformat()

# Set the tickers
tickers = ["ABB", "TSLA", "AMD", "RIOT"]

# Set timeframe to '1D' 
timeframe = "1D"

# Get current closing prices for SPY and AGG
df_tickers = api.get_barset(
    tickers,
    timeframe,
    start=start_date,
    end=end_date
).df
df_tickers_closing_prices= df_tickers.xs("close", axis='columns', level=1)
df_tickers_closing_prices.head()

,ABB,AMD,RIOT,TSLA
time,,,,
2020-01-02 00:00:00-05:00,24.37,49.11,1.217,430.2400
2020-01-03 00:00:00-05:00,24.17,48.57,1.190,442.7500
2020-01-06 00:00:00-05:00,24.32,48.39,1.290,451.4001
2020-01-07 00:00:00-05:00,24.28,48.24,1.485,469.1700
2020-01-08 00:00:00-05:00,24.36,47.84,1.350,492.0900


In [90]:
# Fetch the closing prices 
df_tickers_closing_prices["ABB"] = df_tickers["ABB"]["close"]
df_tickers_closing_prices["TSLA"] = df_tickers["TSLA"]["close"]
df_tickers_closing_prices["AMD"] = df_tickers["AMD"]["close"]
df_tickers_closing_prices["RIOT"] = df_tickers["RIOT"]["close"]

df_tickers_closing_prices["ABB"].head()

C:\Users\jexib\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\jexib\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jexib\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

time
2020-01-02 00:00:00-05:00    24.37
2020-01-03 00:00:00-05:00    24.17
2020-01-06 00:00:00-05:00    24.32
2020-01-07 00:00:00-05:00    24.28
2020-01-08 00:00:00-05:00    24.36
Name: ABB, dtype: float64

In [91]:
# Define Panel Visualization Functions
def robintrack_ABB_stock_price ():
    """Robintrack AAB's accounts effect on ABB's Stock price."""
    abb_rt_data_mean = abb_rt_data.mean()
    #abb_rt_data_mean['users_holding'] = abb_rt_data['users_holding'].fillna(2147.00)
    df_tickers_closing_prices= df_tickers.xs("close", axis='columns', level=1)
    df_tickers_closing_prices_abb = df_tickers_closing_prices["ABB"]
    closing_prices_abb = df_tickers_closing_prices_abb.index.date
    daily_return_abb = closing_prices_abb.pct_change()
    concat_robin_ticker = pd.concat([df_tickers_closing_prices_abb,  abb_rt_data], axis="columns", join="inner")
    #daily_ret_abb = concat_robin_ticker.pct_change()
    abb_corr_plot = concat_robin_ticker.hvplot(title='RobinTrack ABB Accounts Effect in ABB Stock Price')
    return abb_corr_plot
    

    

# Panel Dashboard

In [92]:
abb_analysis= pn.Row("## ABB RobinTrack and Stock Price Analysis", robintrack_ABB_stock_price ())

# Create tabs

robintrack_stock_analysis_dashboard = pn.Tabs("ABB Analysis", abb_analysis)

dashboard= pn.Column(pn.Row("Effect of Number of Individual Robinhood on Unerlying Price & Option Volatility for RIOT, AMD, ABB, and TSLA"), 
                     robintrack_stock_analysis_dashboard)

AttributeError: 'numpy.ndarray' object has no attribute 'pct_change'

# Serve the Panel Dashboard

In [55]:
dashboard.servable()

Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs(active=1)
        [0] Markdown(str)
        [1] Row
            [0] Markdown(str)
            [1] HoloViews(NdOverlay)